## Consumer Loan Default Prediction 



Consumer Loan Default Prediction is a trainable algorithm based on machine learning to identify the loan defaulters. Predicting defaulters would be the primary interest of any lender before lending money. This solution predicts the defaulters and provides the probability score from new borrowers based on previously trained data.  Since predicting defaulters is complex and labour intensive, this solution offers an upper hand to any loan provider to foresee or evaluate any borrower before or during the loan period. 



This sample notebook shows you how to deploy Consumer Loan Default Prediction using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

#### Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to Incident Forecast for IT Infrastructure. If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
    3. Make sure to install below mentioned python libraries before running this notebook
    
#### Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Usage Instruction](#2.-Usage-Instruction)
3. [Training](#3.-Training) 
4. [Sample Data](#4.-Sample-Data)
5. [Perform batch inference](#5.-Perform-batch-inference)
6. [Create an endpoint and perform real-time inference](#6.-Create-an-endpoint-and-perform-real-time-inference)
7. [Clean-up](#7.-Clean-up)
    

#### Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

### 1. Subscribe to the model package

To subscribe to the model package:
1. Open the algorithm listing page **Consumer Loan Default Prediction**
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

### 2. Usage Instruction

The deployed solution has these 2 steps:
1: The system trains on user provided historical incident data and builds & saves a deep learning model which is a representation of the historical data.
2: Once the model is generated, the solution can be used to predict on any new data as input file. 
To achieve this end, the solution deploys the following 2 APIs over AWS Sagemaker:

1.Training API: The system trains on user provided historical process data with relevant features and builds & saves a classification model.
2.Testing API: Once the model is generated, the solution can be used to predict the defaulters for a given test data. 


## Input
Supported content types: `text/csv`
* Following is the mandatory inputs for the APIs:
 * The Target variable should be named** not_default**, this features tells us if someone would default the loan or not.
 

Note: 
1.	Two separate csv input files are required for training and testing.
  * Name of the training file should be Train_Data.csv
  * Name of the testing file should be sample_input.csv
  
## Output
• Content type: ` text/csv`
* Solution will generate an output csv file which will contain 2 additional columns as the prediction along with the columns in the new data.
*  The additional columns will be **Label** and **Score**.

## Invoking endpoint

### AWS CLI Command
If you are using real time inferencing, please create the endpoint first and then use the  following command to invoke it:

``` bash 
!aws sagemaker-runtime invoke-endpoint --endpoint-name $model_name --body fileb://$file_name --content-type 'text/csv' --region us-east-2 output.csv
```
Substitute the following parameters:
* `"model-name"` - name of the inference endpoint where the model is deployed
* `file_name` - input csv file name 
* `text/csv` - content type of the given input 
* `output.csv` - filename where the inference results are written to.

In [ ]:
import base64 
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
import boto3
from IPython.display import Image
from PIL import Image
import numpy as np
import pandas as pd
import cv2
from numpy import asarray

In [ ]:
role = get_execution_role()

sagemaker_session = sage.Session()

bucket=sagemaker_session.default_bucket()
bucket

In [ ]:
# S3 prefixes
common_prefix = "Consumer_Loan_Default_Prediction"
training_input_prefix = common_prefix + "/training-input-data"
batch_inference_input_prefix = common_prefix + "/batch-inference-input-data"

In [ ]:
sagemaker_session = sage.Session()

In [ ]:
# Maintaine this directory structure
TRAINING_WORKDIR = "data/training"

TRAINING_DATA = TRAINING_WORKDIR + "/Train_Data.csv"

In [ ]:
TRAINING_WORKDIR = "data/training"

training_input = sagemaker_session.upload_data(TRAINING_WORKDIR, key_prefix=training_input_prefix)
print ("Training Data Location " + training_input)

### 3. Training 

In [ ]:
import json
import time
from sagemaker.algorithm import AlgorithmEstimator

##### Algorithm ARN

In [ ]:
algorithm_arn ='' # put your ARN here

In [ ]:
algo = AlgorithmEstimator(
    algorithm_arn=algorithm_arn,
    role=role,
    instance_count=1,
    instance_type='ml.m5.12xlarge',
    base_job_name='Consumer-Loan-Default-Prediction')

In [ ]:
print ("Now run the training job using algorithm arn %s in region %s" % (algorithm_arn, sagemaker_session.boto_region_name))
algo.fit({'training': training_input})

### 4. Sample Data

In [ ]:
# maintaine this directory structure
import os
TRANSFORM_WORKDIR = "data/transform"
sample_data = pd.read_csv(TRANSFORM_WORKDIR + "/sample_input.csv",index_col=0)
sample_data.head(10)

### 5. Perform batch inference

In this section, you will perform batch inference using multiple input payloads together. If you are not familiar with batch transform, and want to learn more, see these links:
1. [How it works](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-batch-transform.html)
2. [How to run a batch transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)

In [ ]:
TRANSFORM_WORKDIR = "data/transform"
transform_input = sagemaker_session.upload_data(TRANSFORM_WORKDIR, key_prefix=batch_inference_input_prefix) + "/sample_input.csv"
print("Transform input uploaded to " + transform_input)

In [ ]:
transformer = algo.transformer(1, 'ml.m5.12xlarge')
transformer.transform(transform_input, content_type='text/csv')
transformer.wait()

print("Batch Transform output saved to " + transformer.output_path)

#### Inspect the Batch Transform Output in S3

In [ ]:
from urllib.parse import urlparse

parsed_url = urlparse(transformer.output_path)
bucket_name = parsed_url.netloc
file_key = '{}/{}.out'.format(parsed_url.path[1:], 'sample_input.csv')

s3_client = sagemaker_session.boto_session.client('s3')

response = s3_client.get_object(Bucket = sagemaker_session.default_bucket(), Key = file_key)
response_bytes = response['Body'].read().decode('utf-8')

In [ ]:
bucketFolder = transformer.output_path.rsplit('/')[3]

In [ ]:
import boto3
s3_conn = boto3.client("s3")
bucket_name=sagemaker_session.default_bucket()
with open('output.csv', 'wb') as f:
    s3_conn.download_fileobj(bucket_name, bucketFolder+'/' + 'sample_input.csv' +'.out', f)
    print("Output file loaded from bucket")

### output

In [ ]:
#OUTPUT_WORKDIR = "data/output"

import pandas as pd

out_csv = pd.read_csv("output.csv",index_col=0)
out_csv.head(10)

### 6. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [ ]:
model_name='Consumer-Loan-Default-Prediction'

content_type='text/csv'

real_time_inference_instance_type='ml.m5.12xlarge'
batch_transform_inference_instance_type='ml.m5.12xlarge'

##### Algorithm ARN

In [ ]:
algorithm_arn ='' # put your ARN here

#### A. Create an endpoint

In [ ]:
def predict_wrapper(endpoint, session):
    return sage.RealTimePredictor(endpoint, session,content_type)

#create a deployable model from the model package.
model = ModelPackage(role=role,
                    model_package_arn=algorithm_arn,
                    sagemaker_session=sagemaker_session,
                    predictor_cls=predict_wrapper)

#Deploy the model
predictor = algo.deploy(1, 'ml.m5.12xlarge',endpoint_name=model_name)

Once endpoint has been created, you would be able to perform real-time inference.

#### B. Create input payload

In [ ]:
import pandas as pd
file_name = 'data/transform/sample_input.csv'
data_frame = pd.read_csv(file_name)

#### C. Perform real-time inference

In [ ]:
!aws sagemaker-runtime invoke-endpoint \
    --endpoint-name 'Consumer-Loan-Default-Prediction' \
    --body fileb://$file_name \
    --content-type 'text/csv' \
    --region us-east-2 \
    output.csv

#### D. Visualize output

In [ ]:
out_csv = pd.read_csv("output.csv",index_col=0)
out_csv.head(10)

### 7. Clean-up

#### A. Delete the model

In [ ]:
predictor.delete_endpoint()

#### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

